In [1]:
import os
import time
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
import pyswarms as ps
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import ConvLSTM1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

###############################################################################
# 1) Chargement et préparation des données
###############################################################################
def load_and_prepare_data(csv_path, production_column='production', window_size=24):
    """
    Charge le fichier CSV, scale les données, crée des fenêtres (x,y),
    et renvoie les splits (x_train, y_train, x_test, y_test, x_val, y_val).
    """
    df = pd.read_csv(csv_path)
    # Mise à l'échelle
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(df.values)
    
    # Retrouver l'index de la colonne cible
    target_col_idx = df.columns.get_loc(production_column)
    
    # Création des fenêtres
    x, y = [], []
    for i in range(window_size, len(data_scaled)):
        x.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i, target_col_idx])
    x, y = np.array(x), np.array(y)
    
    # Split train/test/val
    # Ici, 80% train, 10% test, 10% val (à adapter si besoin)
    train_split_index = int(0.8 * len(x))
    test_split_index  = int(0.9 * len(x))
    
    x_train, y_train = x[:train_split_index], y[:train_split_index]
    x_test,  y_test  = x[train_split_index:test_split_index], y[train_split_index:test_split_index]
    x_val,   y_val   = x[test_split_index:], y[test_split_index:]
    
    # Adapter la forme pour ConvLSTM1D (on insère un channel dimension)
    x_train_conv = np.expand_dims(x_train, axis=2)
    x_test_conv  = np.expand_dims(x_test, axis=2)
    x_val_conv   = np.expand_dims(x_val, axis=2)
    
    return (x_train_conv, y_train,
            x_test_conv,  y_test,
            x_val_conv,   y_val,
            df)


###############################################################################
# 2) Baseline (e-base) : un simple entraînement avec des hyperparamètres fixes
###############################################################################
def build_baseline_model(input_shape):
    """
    Construit un modèle ConvLSTM basique avec des hyperparamètres
    fixes (par ex. 64 filtres, 64 neurones denses, lr=0.001).
    """
    model = tf.keras.Sequential([
        ConvLSTM1D(filters=64, kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=64, kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=64, activation='relu'),
        Dense(1, activation="linear")
    ], name="baseline_conv_lstm")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss="mae", optimizer=optimizer)
    return model


###############################################################################
# 3) Modèle paramétrable pour PSO
###############################################################################
def build_convlstm_model(lr, filters1, filters2, dense_units, input_shape):
    """
    Construit et compile un modèle ConvLSTM1D avec hyperparamètres modulables.
    """
    model = tf.keras.Sequential([
        ConvLSTM1D(filters=int(filters1), kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=int(filters2), kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=int(dense_units), activation='relu'),
        Dense(1, activation="linear")
    ], name="model_conv_lstm")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer)
    return model

def plot_and_save_analysis(y_test, y_pred, save_dir, dataset_name):
    os.makedirs(save_dir, exist_ok=True)
    
    # 1. Scatter plot (prédictions vs réel)
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.7, color='orange')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label="Idéal (y = ŷ)")
    plt.xlabel("Valeurs réelles (y)")
    plt.ylabel("Prédictions (ŷ)")
    plt.title(f"{dataset_name} - Prédictions vs Réel")
    plt.legend()
    plt.grid(True)
    scatter_path = os.path.join(save_dir, f"{dataset_name}_scatter.png")
    plt.savefig(scatter_path)
    plt.close()

    # 2. Histogramme des erreurs
    errors = y_test - y_pred
    plt.figure(figsize=(8, 4))
    plt.hist(errors, bins=20, color='orange', edgecolor='black')
    plt.title(f"{dataset_name} - Distribution des erreurs")
    plt.xlabel("Erreur (y - ŷ)")
    plt.ylabel("Fréquence")
    plt.grid(True)
    hist_path = os.path.join(save_dir, f"{dataset_name}_hist.png")
    plt.savefig(hist_path)
    plt.close()

    # 3. Courbe temporelle
    plt.figure(figsize=(10, 4))
    plt.plot(y_test, label="Valeurs réelles", linewidth=2)
    plt.plot(y_pred, '--', label="Prédictions")
    plt.title(f"{dataset_name} - Évolution temporelle")
    plt.xlabel("Index")
    plt.ylabel("Valeur")
    plt.legend()
    plt.grid(True)
    curve_path = os.path.join(save_dir, f"{dataset_name}_courbe.png")
    plt.savefig(curve_path)
    plt.close()

    return scatter_path, hist_path, curve_path
###############################################################################
# 4) Entraînement + évaluation (MAE, MSE, R²) + temps d'exécution
###############################################################################
def train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                             epochs=50, batch_size=512, verbose=0):
    """
    Entraîne le modèle, mesure le temps d'entraînement, et renvoie l'historique.
    """
    stop_early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    start_time = time.time()
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        callbacks=[stop_early])
    training_time = time.time() - start_time
    return history, training_time


def inference_time_and_metrics(model, x_test, y_test):
    """
    Calcule le temps d'inférence, puis renvoie MAE, MSE, R².
    """
    start_time = time.time()
    preds = model.predict(x_test)
    inference_time = time.time() - start_time
    
    preds = preds.reshape(-1)
    y_test = y_test.reshape(-1)
    
    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    r2  = r2_score(y_test, preds)
    
    return mae, mse, r2, inference_time


###############################################################################
# 5) Fonction objectif pour PSO
###############################################################################
def make_objective_function(x_train, y_train, x_val, y_val, input_shape):
    """
    Retourne la fonction objectif à passer à l'optimiseur PSO.
    On entraînera rapidement (quelques époques) pour comparer.
    """
    def objective_function(params):
        """
        Les hyperparamètres sont dans l'ordre :
          [lr, filters1, filters2, dense_units]
        """
        n_particles = params.shape[0]
        losses = np.zeros(n_particles)
        
        for i in range(n_particles):
            lr         = params[i, 0]
            filters1   = int(np.round(params[i, 1]))
            filters2   = int(np.round(params[i, 2]))
            dense_units= int(np.round(params[i, 3]))
            
            # Contrôles pour éviter 0 ou < 1
            filters1    = max(filters1, 1)
            filters2    = max(filters2, 1)
            dense_units = max(dense_units, 1)
            
            # Construction du modèle
            model = build_convlstm_model(lr, filters1, filters2, dense_units, input_shape)
            
            # Entraînement sur quelques époques pour l'évaluation PSO (rapide)
            history = model.fit(x_train, y_train,
                                validation_data=(x_val, y_val),
                                epochs=5,   # Petit nombre d'époques pour PSO
                                batch_size=256,
                                verbose=0)
            val_loss = history.history['val_loss'][-1]
            losses[i] = val_loss
        
        return losses
    
    return objective_function


###############################################################################
# 6) Boucle sur les datasets et compilation des résultats
###############################################################################
def run_experiments_on_datasets(
    dataset_paths,
    production_column='production',
    window_size=24,
    epochs_baseline=50,
    epochs_optimized=50
):
    """
    - Pour chaque dataset :
        1) Prépare les données
        2) Entraîne le modèle baseline (e-base) et mesure ses métriques
        3) Lance l'optimisation PSO
        4) Entraîne le modèle avec les hyperparams optimisés
        5) Mesure les métriques et temps
        6) Stocke les résultats dans un DataFrame
    """
    results = []
    
    for csv_path in dataset_paths:
        dataset_name = os.path.basename(csv_path).replace('.csv','')
        print(f"\n=== Dataset: {dataset_name} ===")
        
        # 1) Chargement et préparation
        x_train, y_train, x_test, y_test, x_val, y_val, df = load_and_prepare_data(
            csv_path,
            production_column=production_column,
            window_size=window_size
        )
        input_shape = x_train.shape[1:]
        
        # 2) Modèle baseline
        baseline_model = build_baseline_model(input_shape)
        history_base, t_train_base = train_and_evaluate_model(
            baseline_model, x_train, y_train, x_val, y_val,
            epochs=epochs_baseline, batch_size=512, verbose=0
        )
        mae_base, mse_base, r2_base, t_infer_base = inference_time_and_metrics(baseline_model, x_test, y_test)
        
        # 3) PSO : définition des bornes et optimisation
        #    [lr, filters1, filters2, dense_units]
        lower_bounds = [1e-4, 32, 32, 32]
        upper_bounds = [1e-2, 128,128,128]
        bounds = (np.array(lower_bounds), np.array(upper_bounds))
        
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
        n_particles = 10
        dimensions  = 4
        
        objective_fn = make_objective_function(x_train, y_train, x_val, y_val, input_shape)
        optimizer = ps.single.GlobalBestPSO(n_particles=n_particles,
                                            dimensions=dimensions,
                                            options=options,
                                            bounds=bounds)
        
        start_t_pso = time.time()
        best_cost, best_pos = optimizer.optimize(objective_fn, iters=5)
        pso_time = time.time() - start_t_pso
        
        # 4) Entraîner le modèle avec les hyperparamètres optimisés
        lr_opt        = best_pos[0]
        filters1_opt  = int(np.round(best_pos[1]))
        filters2_opt  = int(np.round(best_pos[2]))
        dense_opt     = int(np.round(best_pos[3]))
        
        best_model = build_convlstm_model(lr_opt, filters1_opt, filters2_opt, dense_opt, input_shape)
        history_opt, t_train_opt = train_and_evaluate_model(
            best_model, x_train, y_train, x_val, y_val,
            epochs=epochs_optimized, batch_size=512, verbose=0
        )
        
        # 5) Évaluation finale
        mae_opt, mse_opt, r2_opt, t_infer_opt = inference_time_and_metrics(best_model, x_test, y_test)
        
        scatter_path, hist_path, curve_path = plot_and_save_analysis(
            y_test=y_test, 
            y_pred=best_model.predict(x_test).flatten(), 
            save_dir="plots", 
            dataset_name=dataset_name
        )
        
        # 6) Stockage des résultats dans un dictionnaire
        result_dict = {
            "Dataset": dataset_name,
            
            # E-base
            "MAE e-base": mae_base,
            "MSE e-base": mse_base,
            "R2 e-base":  r2_base,
            "T(entrainement-e-base)[s]": t_train_base,
            
            # PSO
            "PSO best R2": r2_opt,
            "PSO best cost (val_loss)": best_cost,
            "Paramètres optimisés": f"lr={lr_opt:.5f}, f1={filters1_opt}, f2={filters2_opt}, dense={dense_opt}",
            "T(PSO)[s]": pso_time,
            
            # Entraînement optimisé
            "T(entrainement-optimisé)[s]": t_train_opt,
            "MAE optimisé": mae_opt,
            "MSE optimisé": mse_opt,
            "R2 optimisé": r2_opt,
            "Graph_scatter": scatter_path,
            "Graph_hist": hist_path,
            "Graph_courbe": curve_path,
            
            # Inférence
            "T(evaluation-inference)[s]": t_infer_opt,
            
            # Nombre d'époques
            "Nombre d'époques e-base": epochs_baseline,
            "Nombre d'époques optimisé": epochs_optimized
        }
        
        results.append(result_dict)
    
    # Conversion en DataFrame
    df_results = pd.DataFrame(results)
    return df_results


###############################################################################
# 7) Lancement final
###############################################################################
if __name__ == "__main__":
    # Liste des chemins vers vos CSV
    dataset_paths = [
        "scaled_dataset.csv",
    ]
    
    # Paramètres globaux (à adapter)
    production_column = 'production'
    window_size = 24      # taille des fenêtres
    epochs_baseline = 1  # nombre d'époques pour la baseline
    epochs_optimized = 5 # nombre d'époques pour le modèle optimisé
    
    # Lancement des expériences
    df_results = run_experiments_on_datasets(
        dataset_paths,
        production_column=production_column,
        window_size=window_size,
        epochs_baseline=epochs_baseline,
        epochs_optimized=epochs_optimized
    )
    
    # Affichage des résultats finaux
    print("\n========== RÉSULTATS FINAUX ==========")
    print(df_results)
    # Sauvegarde éventuellement en CSV
    df_results.to_csv("resume_resultats.csv", index=False)


2025-06-16 14:33:38.158484: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 14:33:38.222131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-16 14:33:40.338723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



=== Dataset: scaled_dataset ===


2025-06-16 14:33:41.757711: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-06-16 14:33:41.757773: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:160] env: CUDA_VISIBLE_DEVICES="-1"
2025-06-16 14:33:41.757781: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-06-16 14:33:41.757787: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-06-16 14:33:41.757792: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: ST-RECH-BLOC
2025-06-16 14:33:41.757797: I external/local_xla/xla/stream_executor/cuda/

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step


2025-06-16 14:33:54,680 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|                                                                              |0/5/home/sismail/py_envs/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
pyswarms.single.global_best: 100%|████████████████████████████████████████████████████████████|5/5, best_cost=0.0241
2025-06-16 15:19:20,640 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.024064794182777405, best pos: [7.98232034e-03 3.36459319e+01 6.53240081e+01 7.81262379e+01]


34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step

========== RÉSULTATS FINAUX ==========
          Dataset  MAE e-base  MSE e-base  R2 e-base  \
0  scaled_dataset    0.029263    0.001454  -0.296334   

   T(entrainement-e-base)[s]  PSO best R2  PSO best cost (val_loss)  \
0                  10.869625     0.312563                  0.024065   

                 Paramètres optimisés    T(PSO)[s]  \
0  lr=0.00798, f1=34, f2=65, dense=78  2725.962314   

   T(entrainement-optimisé)[s]  MAE optimisé  MSE optimisé  R2 optimisé  \
0                    29.408428      0.021409      0.000771     0.312563   

                      Graph_scatter                     Graph_hist  \
0  plots/scaled_dataset_scatter.png  plots/scaled_dataset_hist.png   

                      Graph_courbe  T(evaluation-inference)[s]  \
0  plots/scaled_dataset_courbe.png                    3.122226   

   Nombre d'époques e-base  Nombre d'époques optimisé  
0                        1        